# Computing on GPUs

## GPU (local)

First of all, let's check if there is a GPU available.

In [ ]:
using CUDAdrv

In [ ]:
CuDevice(0)

### Matrix multiplication

In [ ]:
A, B = rand(1000,1000), rand(1000,1000);

Let's move these arrays to the GPU.

In [ ]:
using CuArrays
@assert CuArrays.functional() # if this fails your GPU isn't recognized correctly

In [ ]:
Agpu, Bgpu = CuArray(A), CuArray(B);

That's it!

In [ ]:
typeof(Agpu)

How much faster is a simple matmul on the GPU? Let's find out.

In [ ]:
using BenchmarkTools

In [ ]:
println("A*B (cpu)")
@btime $A * $B;

In [ ]:
println("A*B (gpu)")
@btime $Agpu * $Bgpu;

That's at least 3 orders of magnitude faster!

In [ ]:
# Free GPU memory
Agpu, Bgpu = nothing, nothing
GC.gc()

Note that the result of the multiplication lives on the GPU as well and needs to be pulled back to main memory.

In [ ]:
Agpu, Bgpu = CuArray(A), CuArray(B);

In [ ]:
Cgpu = Agpu * Bgpu;

In [ ]:
typeof(Cgpu)

In [ ]:
C = Matrix(Cgpu); # move to cpu

How long does it take to move the `CuArray` back to main memory?

In [ ]:
@btime Matrix($Cgpu);

In [ ]:
# Free GPU memory
Agpu, Bgpu, Cgpu = nothing, nothing, nothing
GC.gc()

### Machine learning

In [ ]:
using Flux

In [ ]:
m = Chain(
    Dense(1000, 100),
    Dense(100, 10),
    Dense(10, 5),
    Dense(5, 2),
    softmax # normalize output neurons
    )

data = rand(1000, 1000); # fake data
labels = fill(0.5, 2, 1000); # fake data

loss(x, y) = sum(Flux.mse(m(x), y)) # mean squared error
opt = Descent(0.01)

In [ ]:
@time Flux.train!(loss, Flux.params(m), [(data,labels)], opt)

Let's train the network on the GPU instead! It's as simple as `|> gpu`:

In [ ]:
# move the model to the gpu
m = Chain(
    Dense(1000, 100),
    Dense(100, 10),
    Dense(10, 5),
    Dense(5, 2),
    softmax
    ) |> gpu

# move data to the gpu
data = rand(1000, 1000) |> gpu;
labels = fill(0.5, 2, 1000) |> gpu;

loss(x, y) = sum(Flux.mse(m(x), y))
opt = Descent(0.01)

In [ ]:
typeof(m)

In [ ]:
@time Flux.train!(loss, Flux.params(m), [(data,labels)], opt)

The training is about **two orders of magnitude faster** on the GPU in this case!

Now that our model is trained, let's feed it some data.

In [ ]:
m(rand(1000))

Oops. Since our model lives on the GPU we can't feed it with data living in main memory. We must move our model back to the CPU first.

In [ ]:
m_cpu = m |> cpu

In [ ]:
typeof(m_cpu)

In [ ]:
m_cpu(rand(1000))

## GPU (remote)

Let's start a worker on a gpu node of a supercomputer cluster.

In [ ]:
using Distributed
addprocs([("cbauer17@gpu2", 1)]; exename=`/projects/ag-trebst/bauer/bin/julia-1.3.1/bin/julia`, exeflags=`--project=/projects/ag-trebst/bauer/JuliaOulu20/backup/gpu`, dir="/projects/ag-trebst/bauer/JuliaOulu20/backup/gpu", tunnel=true)
@fetch gethostname()

In [ ]:
params = (exename=`nice -19 /home/bauer/bin/julia-1.3.1/bin/julia --project=/home/bauer/JuliaOulu20`, dir="/home/bauer")
addprocs([("l94", :auto)]; params...)

In [ ]:
using Distributed
addprocs([("cbauer17@gpu2", 1)]; exename=`/projects/ag-trebst/bauer/bin/julia-1.3.1/bin/julia`, dir=`/projects/ag-trebst/bauer`, tunnel=true)
@fetch gethostname()

Extract the GPU name:

In [ ]:
@fetch @eval using CUDAdrv
@fetch CUDAdrv.name(CuDevice(0))

### Matrix multiplication

In [ ]:
@fetch @eval using CuArrays, BenchmarkTools

In [ ]:
@fetch begin
    A, B = rand(1000,1000), rand(1000,1000);
    Agpu, Bgpu = CuArray(A), CuArray(B);
    
    println("Move array CPU -> GPU")
    @btime CuArray($A);
    
    println("A*B (cpu)")
    @btime $A * $B;

    println("A*B (gpu)")
    @btime $Agpu * $Bgpu;
    
    
    println("Move array GPU -> CPU")
    Cgpu = Agpu * Bgpu
    @btime Array($Cgpu);
    
    nothing
end